In [1]:
import sys
sys.path.insert(0,'../../backend')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math

import gym
import stable_baselines
from stable_baselines import A2C, ACER, ACKTR, DQN, DDPG, SAC, PPO1, PPO2, TD3, TRPO
from stable_baselines.common.env_checker import check_env
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.gail import ExpertDataset, generate_expert_traj

import webotsgym as wg

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
from webotsgym.utils import exponential_decay
from webotsgym.env.reward import step_pen_exp

class MyEval(wg.WbtReward):
    def __init__(self, env, config):
        super(MyEval, self).__init__(env, config)

    def calc_reward(self):
        target_distance = self.env.get_target_distance(False)
        if target_distance < 0.1:
            return 5000 + 5000 * (1 - abs(self.env.state.speed))
        else:
            reward = 0
            reward += -2
            
            if len(self.env.distances) > 0 and len(self.env.history) > 1:
                target_diff = self.env.distances[-1] - self.env.get_target_distance(False)
                
                gps_diff_0 = self.env.history[-2].gps_actual[0] - self.env.state.gps_actual[0]
                gps_diff_1 = self.env.history[-2].gps_actual[1] - self.env.state.gps_actual[1]
                
                #print("history: ", self.env.history[-1].gps_actual[0])
                #print("actual:  " , self.env.state.gps_actual[0])
                
                actual_diff = math.sqrt(gps_diff_0**2 + gps_diff_1**2)
                
                if actual_diff > 0:
                    diff_rew = 4 * ( target_diff  / actual_diff)
                else:
                    diff_rew = 0
                
                
                
                if abs(diff_rew) > 4:
                    print("============= Quotient > 1 !? ==========")
                    print("Target_diff: ", target_diff)
                    print("Actual_diff: ", actual_diff)
                    print("diff_rew:    ", diff_rew)
                    
                if diff_rew < 0:
                    diff_rew = diff_rew * 2
                
                #print("Target_diff: ", target_diff)
                #print("Actual_diff: ", actual_diff)
                #print("diff_rew:    ", diff_rew)  
                    
  
                reward += diff_rew
                    
            if self.env.state.action_denied:
                reward += -5
        
            if self.env.state.touching:
                reward += -100
        return reward

    def check_done(self):
        if self.env.total_reward < -5000:
            print("reward boundary, reward: ", self.env.total_reward)
            return True
        if self.env.get_target_distance(False) < 0.1:
            print("target reached, reward: ", self.env.total_reward)
            return True
        if self.env.total_reward > 25000:
            print("Stop hacking!, reward: ", self.env.total_reward)
            return True
        return False

In [3]:
config = wg.WbtConfig()
config.world_size = 3
config.num_obstacles = 0
config.sim_mode = wg.config.SimSpeedMode.FAST
config.sim_step_every_x = 10
config.relative_action = True
config.direction_type = wg.config.DirectionType.STEERING

In [4]:
env = wg.WbtGym(train=True, 
                evaluate_class=MyEval,
#                 action_class = wg.WbtActDiscrete(config, dspeed=0.05, ddir=0.05),
                config=config)

../../backend/webotsgym/env/webotenv.py:111: UserWarning: Relative property of action class is overwritten by config.relative_action.
  warnings.warn("Relative property of action class is overwritten "


Accepting on Port:  10201


In [5]:
model_name = "3x3_reach_target_v6"
model = PPO1("MlpPolicy", env, timesteps_per_actorbatch = 5000, tensorboard_log="./{}".format(model_name))





Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.









Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
while True:
    model.learn(total_timesteps=500000)
    model.save("./{}".format(model_name))


reward boundary, reward:  -5075.138390909345
reward boundary, reward:  -5013.254815767437
reward boundary, reward:  -5033.5091386599715
reward boundary, reward:  -5087.786258716982
reward boundary, reward:  -5024.045613093157
reward boundary, reward:  -5008.509655753781
reward boundary, reward:  -5054.38617389161
reward boundary, reward:  -5080.072414041448
reward boundary, reward:  -5100.005598459242
reward boundary, reward:  -5067.950279860842
reward boundary, reward:  -5011.904305615229
target reached, reward:  -1769.3919054887645
target reached, reward:  -1506.5026336757157

reward boundary, reward:  -5001.076360459701
reward boundary, reward:  -5003.74068047095
target reached, reward:  -2775.7471875119295
reward boundary, reward:  -5075.205207944889
reward boundary, reward:  -5104.994773197897
target reached, reward:  -4490.768425756203
reward boundary, reward:  -5004.264622656174
reward boundary, reward:  -5019.834311626317
target reached, reward:  -1318.1083992078989
reward bou

target reached, reward:  -1801.6668112116356
reward boundary, reward:  -5005.370643129012
target reached, reward:  -26.13245404180086
target reached, reward:  10.723849348179156
reward boundary, reward:  -5005.429028366687
reward boundary, reward:  -5037.600346670333
reward boundary, reward:  -5096.891715493432
reward boundary, reward:  -5031.034926587288
target reached, reward:  -1406.3511001691481
reward boundary, reward:  -5059.31741492684
target reached, reward:  -1807.6137520876023
reward boundary, reward:  -5020.189858048437
target reached, reward:  -2155.6087686127294
target reached, reward:  -2306.0624475616446
target reached, reward:  -579.3767082489529
reward boundary, reward:  -5065.743816013294
reward boundary, reward:  -5114.091098103273
target reached, reward:  -3516.1488047165
reward boundary, reward:  -5078.493407652511
target reached, reward:  -1663.1247570877745
reward boundary, reward:  -5038.911222730515
reward boundary, reward:  -5097.523006580834
reward boundary, 

target reached, reward:  -404.78630738889063
target reached, reward:  -3626.146026449923
reward boundary, reward:  -5068.249716901806
reward boundary, reward:  -5094.439563591431
reward boundary, reward:  -5001.164689756327
target reached, reward:  -396.18692134920315
reward boundary, reward:  -5043.858754286443
reward boundary, reward:  -5001.946080106634
reward boundary, reward:  -5065.990288523952
reward boundary, reward:  -5106.4298029542115
reward boundary, reward:  -5006.0869265811925
reward boundary, reward:  -5076.244678475422
reward boundary, reward:  -5031.7835877621055
target reached, reward:  -2504.6359978153573
target reached, reward:  -650.3531072213019
reward boundary, reward:  -5101.3312892827325
reward boundary, reward:  -5105.296879360163
target reached, reward:  -1272.8959776039326
reward boundary, reward:  -5008.024838641855
target reached, reward:  -3638.1026696904296
target reached, reward:  -319.843437282465
reward boundary, reward:  -5002.841719706121
reward bou

reward boundary, reward:  -5089.840973514016
reward boundary, reward:  -5068.378066294903
reward boundary, reward:  -5045.20640640242
reward boundary, reward:  -5069.358816819288
target reached, reward:  -2605.5771348124945
reward boundary, reward:  -5021.852340242994
reward boundary, reward:  -5007.013325019218
target reached, reward:  -2325.2534240048703
reward boundary, reward:  -5033.391350495466
reward boundary, reward:  -5000.952209513224
target reached, reward:  -3375.478657891901
target reached, reward:  -3785.9308670610403
reward boundary, reward:  -5039.519366929573
reward boundary, reward:  -5066.774540043155
reward boundary, reward:  -5033.276754229783
reward boundary, reward:  -5087.814878958694
reward boundary, reward:  -5065.32587073747
target reached, reward:  -21.476678026662988
reward boundary, reward:  -5060.617355874746
target reached, reward:  -2534.0623620984106
reward boundary, reward:  -5007.776203096292
reward boundary, reward:  -5093.2372970949355
target reach

reward boundary, reward:  -5006.654415675706
reward boundary, reward:  -5002.681955768866
target reached, reward:  -54.0221963039583
target reached, reward:  9.7146736735573
reward boundary, reward:  -5005.637823516773
target reached, reward:  -3355.481866093095
reward boundary, reward:  -5067.399543224968
target reached, reward:  -411.07328780143547
reward boundary, reward:  -5022.615692187488
reward boundary, reward:  -5095.158920838438
reward boundary, reward:  -5002.894794551213
reward boundary, reward:  -5008.070459989069
reward boundary, reward:  -5054.319170931672
reward boundary, reward:  -5039.632489386065
reward boundary, reward:  -5000.981605191577
reward boundary, reward:  -5000.607996560507
reward boundary, reward:  -5003.93068100558
target reached, reward:  -2820.5023356756556
reward boundary, reward:  -5005.571623792463
target reached, reward:  -2069.001544638916
reward boundary, reward:  -5071.730126238841
reward boundary, reward:  -5008.73854181445
reward boundary, rew

...

target reached, reward:  -837.9573540438486
target reached, reward:  -381.9672030372562
reward boundary, reward:  -5018.752943277807
target reached, reward:  -55.410830320070566
target reached, reward:  -811.7338634578784
target reached, reward:  0
target reached, reward:  -68.16484945065773
target reached, reward:  -655.1050049167546
target reached, reward:  -766.2546966591113
target reached, reward:  -169.21066656732924
target reached, reward:  -737.1007772382094
target reached, reward:  -148.45838686811214
target reached, reward:  -45.0984027835207
reward boundary, reward:  -5003.494951112983
target reached, reward:  0
target reached, reward:  -124.75559139256696
target reached, reward:  -16.98262587096754
target reached, reward:  -8.72678184630303
target reached, reward:  -158.69542454540326
target reached, reward:  -15.284521972630001
target reached, reward:  -277.98245541906203
target reached, reward:  0
target reached, reward:  -45.29572268790578
reward boundary, reward:  -5015.

target reached, reward:  -97.81880558886074
target reached, reward:  -52.59247178928337
target reached, reward:  -318.18989952023213
target reached, reward:  -62.28964253451495
target reached, reward:  -2.8525338824450155
target reached, reward:  -254.29621636730508
target reached, reward:  -116.95482830294333
target reached, reward:  0
target reached, reward:  -178.38094078456552
target reached, reward:  7.756417777499083
target reached, reward:  0
target reached, reward:  -591.7850646278268
target reached, reward:  -14.741772825707994
target reached, reward:  -382.5196156175498
target reached, reward:  0
target reached, reward:  0
target reached, reward:  -2367.4663206888176
target reached, reward:  -585.3603995670228
target reached, reward:  -3572.9133532466235
target reached, reward:  -497.9801924197522
target reached, reward:  -42.57119161403736
target reached, reward:  -3283.483699886124
target reached, reward:  -150.9121564469078
target reached, reward:  -4697.822030961449
targe

target reached, reward:  0
target reached, reward:  -12.700177430944638
target reached, reward:  0
target reached, reward:  -25.101959173494183
target reached, reward:  -51.30118945129168
target reached, reward:  -232.6304530814881
target reached, reward:  -27.939066787071827
target reached, reward:  -699.5546281604167
target reached, reward:  3.055366201197362
target reached, reward:  0
target reached, reward:  -2068.7740635006335
target reached, reward:  -0.529905662762272
target reached, reward:  0
target reached, reward:  0
target reached, reward:  -847.4728132259017
target reached, reward:  -54.50417746928979
target reached, reward:  -1098.6764112239634
target reached, reward:  0
target reached, reward:  -7.139667339195254
target reached, reward:  -344.62908729623217
target reached, reward:  -635.8498542340411
target reached, reward:  -106.32299344105573
target reached, reward:  0
target reached, reward:  -154.0556030331028
reward boundary, reward:  -5001.18163743491
target reache

target reached, reward:  -423.2374449277014
target reached, reward:  -107.97216787004415
target reached, reward:  -38.08170346656131
target reached, reward:  -6.7425890301486815
target reached, reward:  -22.36782174156836
target reached, reward:  -4.08091030057145
target reached, reward:  0
reward boundary, reward:  -5003.853755693489
target reached, reward:  -8.111339241673685
target reached, reward:  -382.95896282362673
target reached, reward:  0
target reached, reward:  -94.10141213208995
target reached, reward:  0
target reached, reward:  -51.335319004617034
target reached, reward:  -2650.219375106082
target reached, reward:  -41.52989640126795
target reached, reward:  8.233145059123956
target reached, reward:  -1523.9405712028256
target reached, reward:  -422.02371374832103
target reached, reward:  -2724.9649647897077
target reached, reward:  -88.53855885124258
target reached, reward:  -162.24281475657412
target reached, reward:  -1001.9677991176454
target reached, reward:  -403.4

target reached, reward:  -16.36857080959856
target reached, reward:  -69.74984261631181
target reached, reward:  -142.94373258236155
target reached, reward:  -28.31299036035488
target reached, reward:  -4121.130980409734
target reached, reward:  -16.34351887191879
target reached, reward:  -2681.190831969085
target reached, reward:  -22.27607670314578
target reached, reward:  -239.32243496109714
target reached, reward:  -19.540413583686718
target reached, reward:  -48.33172460735026
target reached, reward:  -40.62008824095103
target reached, reward:  -135.23939262870257
target reached, reward:  2.8275716236806074
target reached, reward:  -327.47712108996353
target reached, reward:  -10.333161788315163
target reached, reward:  -100.87211256430611
target reached, reward:  -21.95393135361639
target reached, reward:  0
target reached, reward:  -501.21544339126933
target reached, reward:  -39.627379862891196
target reached, reward:  -78.23370480844595
target reached, reward:  -899.1182698833